# Naive Bayes

# Import Libraries

In [1]:
!pip install nltk

In [2]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]    |

True

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

from collections import Counter

# Import File

In [4]:
df = pd.read_csv("data/TwitterDataset.csv", encoding='latin')

In [5]:
df.head()

,File Name,Caption,LABEL
0,1.txt,How I feel today #legday #jelly #aching #gym,negative
1,10.txt,@ArrivaTW absolute disgrace two carriages from...,negative
2,100.txt,This is my Valentine's from 1 of my nephews. I...,positive
3,1000.txt,betterfeelingfilms: RT via Instagram: First da...,neutral
4,1001.txt,Zoe's first love #Rattled @JohnnyHarper15,positive


In [6]:
df.drop(columns=["File Name"], inplace=True)
df.dropna(inplace=True)

In [7]:
df["LABEL"].value_counts()

LABEL
neutral     1771
positive    1646
negative    1452
Name: count, dtype: int64

In [8]:
df = df[~df['LABEL'].isin(['neutral'])]
df.reset_index(drop=True, inplace=True)

In [9]:
df.sample(100)

,Caption,LABEL
155,"Eyes closed tonight for my Pilates, taking me ...",positive
2622,RT @NWSLosAngeles: Isolated showers and thunde...,negative
2741,RT @khununeos: khun was literally me whenever ...,negative
620,Meet Sir Button available #Compassionate #Adop...,positive
2379,"So, apparently my liver enzymes are elevated. ...",negative
...,...,...
2758,Rt GrootBot: I am Groot. I_AM__GROOT_: RT What...,negative
2000,RT @KimKardashian: These wild ladies always he...,negative
1406,RT @SavageOnIy: Look how cute this chicken is ??,positive
2541,"coniks2: RT ClevelandChick: Happy birthday, #C...",positive


In [10]:
df["LABEL"].value_counts()

LABEL
positive    1646
negative    1452
Name: count, dtype: int64

In [11]:
df['LABEL'] = df['LABEL'].map({'negative':0, 'positive':1})

# Preprocess

In [12]:
def remove_usernames(text):
    return re.sub(r'@\w+', '', text)

def remove_hashtags(text):
    return re.sub(r'#', '', text)

def clean_text(text):
    # Removing URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Removing special characters and numbers
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', '', text)
    # Removing extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize_text(text):
    return word_tokenize(text)

def case_folding(text):
    return text.lower()

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

def stem_tokens(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in tokens]

def preprocess_text(text):
    text = remove_usernames(text)
    text = remove_hashtags(text)
    text = clean_text(text)
    text = case_folding(text)
    tokens = tokenize_text(text)
    tokens = remove_stopwords(tokens)
    tokens = stem_tokens(tokens)
    return ' '.join(tokens)

In [13]:
text = df['Caption'][0]
print(text)
print(preprocess_text(text))

How I feel today #legday #jelly #aching #gym 
feel today legday jelli ach gym


# Preprocess

In [14]:
X = df['Caption'].apply(preprocess_text)
y = df['LABEL'].values

# Split

In [15]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=11)

# Helper Functions

In [16]:
def count_tweets(result, tweets, ys):

    for y, tweet in zip(ys, tweets):
        for word in tweet.split():
            # define the key, which is the word and label tuple
            pair = (word,y)

            # if the key exists in the dictionary, increment the count
            if pair in result:
                result[pair] += 1

            # else, if the key is new, add it to the dictionary and set the count to 1
            else:
                result[pair] = 1

    return result

In [17]:
freqs = count_tweets({}, x_train, y_train)
freqs

{('reason', 0): 4,
 ('petrifi', 0): 18,
 ('ocean', 0): 3,
 ('confess', 0): 1,
 ('rt', 0): 505,
 ('one', 0): 24,
 ('await', 0): 1,
 ('trial', 0): 2,
 ('jail', 0): 2,
 ('simpli', 0): 1,
 ('could', 0): 4,
 ('afford', 0): 1,
 ('bail', 0): 1,
 ('rt', 1): 597,
 ('passion', 1): 62,
 ('post', 1): 8,
 ('top', 1): 23,
 ('market', 1): 5,
 ('love', 1): 165,
 ('content', 1): 1,
 ('ufc', 0): 5,
 ('loui', 0): 3,
 ('smolka', 0): 1,
 ('irish', 0): 1,
 ('fan', 0): 8,
 ('belliger', 0): 4,
 ('go', 0): 30,
 ('australia', 0): 3,
 ('open', 0): 1,
 ('chri', 0): 4,
 ('roger', 0): 2,
 ('miss', 0): 15,
 ('train', 0): 7,
 ('today', 0): 25,
 ('check', 0): 3,
 ('dizzi', 0): 9,
 ('spell', 0): 3,
 ('second', 0): 8,
 ('test', 0): 3,
 ('lord', 0): 4,
 ('ssnhq', 0): 1,
 ('dog', 0): 19,
 ('watch', 0): 9,
 ('other', 0): 3,
 ('tortur', 0): 17,
 ('daylight', 0): 1,
 ('samsung', 0): 1,
 ('sponsor', 0): 2,
 ('guid', 0): 3,
 ('hear', 0): 6,
 ('stopbokn', 0): 6,
 ('thank', 1): 53,
 ('u', 1): 20,
 ('understand', 1): 2,
 ('heart'

In [18]:
def lookup(freqs, word, label):
    # Return frequency of word and corresponding label
    return freqs.get((word, label), 0)

In [19]:
lookup(freqs,'sad',0) 

14

# Priors and LogLikehood

Here’s how you can represent the mathematical formulas for the log prior probability and log likelihood in Python Markdown using LaTeX:

Log Prior Probability
The log prior probability can be expressed as:

markdown
Copy code
$$
\text{logprior} = \log \left( \frac{D_{pos}}{D_{neg}} \right)
$$
Where:

𝐷
𝑝
𝑜
𝑠
D 
pos
​
  is the number of positive documents.
𝐷
𝑛
𝑒
𝑔
D 
neg
​
  is the number of negative documents.
Log Likelihood
The log likelihood for a word 
𝑤
w can be expressed as:

markdown
Copy code
$$
\text{loglikelihood}(w) = \log \left( \frac{P(w \mid \text{Positive})}{P(w \mid \text{Negative})} \right)
$$
Where:

𝑃
(
𝑤
∣
Positive
)
P(w∣Positive) is the probability of word 
𝑤
w given a positive document.
𝑃
(
𝑤
∣
Negative
)
P(w∣Negative) is the probability of word 
𝑤
w given a negative document.
With additive smoothing, these probabilities are calculated as:

markdown
Copy code
$$
P(w \mid \text{Positive}) = \frac{\text{freq}_{pos}(w) + 1}{N_{pos} + V}
$$
$$
P(w \mid \text{Negative}) = \frac{\text{freq}_{neg}(w) + 1}{N_{neg} + V}
$$
Thus, the log likelihood is:

markdown
Copy code
$$
\text{loglikelihood}(w) = \log \left( \frac{\frac{\text{freq}_{pos}(w) + 1}{N_{pos} + V}}{\frac{\text{freq}_{neg}(w) + 1}{N_{neg} + V}} \right)
$$
$$
\text{loglikelihood}(w) = \log (\text{freq}_{pos}(w) + 1) - \log (\text{freq}_{neg}(w) + 1) + \log \left( \frac{N_{neg} + V}{N_{pos} + V} \right)
$$

In [20]:
def get_logprior(y_train):

    # Calculate D, D_pos, and D_neg
    D_pos = sum(1 for label in y_train if label == 1)
    D_neg = len(y_train) - D_pos  # Total documents - D_pos

    # Calculate logprior
    logprior = np.log(D_pos / D_neg)

    return logprior

def get_loglikelihood(freqs):
    loglikelihood = {}
    
    # Extract vocabulary and counts directly
    vocab = set(word for word, _ in freqs.keys())
    
    # Calculate N_pos, N_neg, V_pos, V_neg
    pos_counts = Counter()
    neg_counts = Counter()
    
    for (word, label), count in freqs.items():
        if label == 1:
            pos_counts[word] += count
        else:
            neg_counts[word] += count
    
    N_pos = sum(pos_counts.values())
    N_neg = sum(neg_counts.values())
    V = len(vocab)

    # Calculate loglikelihood
    for word in vocab:
        freq_pos = pos_counts[word]
        freq_neg = neg_counts[word]

        # Calculate probabilities with additive smoothing
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # Calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos / p_w_neg)
    
    return loglikelihood

In [21]:
logprior = get_logprior(y_train)
loglikelihood = get_loglikelihood(freqs)

In [22]:
print(logprior)
print(loglikelihood)

0.14391210867908333
{'khaderadnan': 0.5535986511931195, 'enthusiast': 2.9959456865623237, 'rancher': 0.5535986511931195, 'smh': -0.8326957099267712, 'median': -0.8326957099267712, 'desk': -1.2381608180349357, 'soston': -0.8326957099267712, 'gloriou': 0.5535986511931195, 'stain': -0.8326957099267712, 'clinch': 0.5535986511931195, 'bball': -0.8326957099267712, 'pathet': -0.8326957099267712, 'domest': -0.8326957099267712, 'shark': -0.42723060181860684, 'lovebird': 0.5535986511931195, 'gust': 0.5535986511931195, 'n': -0.7149126742703878, 'defo': 0.5535986511931195, 'dimens': 0.5535986511931195, 'goer': -0.8326957099267712, 'srkian': 0.5535986511931195, 'alon': -0.42723060181860684, 'yyc': -0.8326957099267712, 'unprincipl': 0.5535986511931195, 'bakhtishu': 0.5535986511931195, 'macktruck': -0.8326957099267712, 'longtailedtit': 0.5535986511931195, 'struck': -0.8326957099267712, 'â': -0.13954852936682588, 'superpod': 0.5535986511931195, 'rss': -0.8326957099267712, 'theborgia': 0.55359865119311

In [23]:
loglikelihood['sad']

-1.7489864418009262

# Now predict


### Prediction Example

Let’s classify the tweet: "I am very happy".

1. **Preprocess**: Extract words from the tweet:

   $$ \text{words} = \text{['i', 'am', 'very', 'happy']} $$

2. **Compute Log Probability**:

   Assuming the log prior and log likelihoods are:

   - **Log Prior**: \( \text{logprior} = \log (1.5) \)
   - **Log Likelihood for "happy"**: \( \text{loglikelihood}(\text{"happy"}) = \log (4) \)

   The log probability of the tweet being positive is:

   $$ \text{log\_prob} = \text{logprior} + \text{loglikelihood}(\text{"happy"}) $$

   Substituting the values:

   $$ \text{log\_prob} = \log (1.5) + \log (4) = \log (1.5 \times 4) = \log (6) $$

### Interpretation

- **Positive Log Probability**: If \( \text{log\_prob} \) is positive, it indicates the evidence is stronger for the tweet being positive.

- **Negative Log Probability**: If \( \text{log\_prob} \) were negative, it would suggest the evidence is stronger for the tweet being negative.

### Summary

- **Positive Log Probability**:

  $$ \text{log\_prob} > 0 \implies \text{Stronger evidence for the Positive Class} $$

- **Negative Log Probability**:

  $$ \text{log\_prob} < 0 \implies \text{Stronger evidence for the Negative Class} $$

By comparing the log probabilities for both classes, you can determine the final classification of the tweet.


In [24]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    
    # Preprocess the tweet to get a list of words
    words = preprocess_text(tweet)
    
    # Initialize the log probability with the log prior
    log_prob = logprior
    
    # Add the log likelihood of each word if it exists in the dictionary
    for word in words.split():
        log_prob += loglikelihood.get(word,0)
        # print(word,loglikelihood.get(word,0))
    return log_prob

# Test

In [25]:
# my_tweet = '@ArrivaTW absolute disgrace two carriages from Bangor half way there standing room only #disgraced'
my_tweet = "I was feeling happy but now I am sad"
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The output is', p)

The output is 0.9504415325551443


In [26]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    accuracy = 0  # return this properly

    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)


    error = np.mean(np.absolute(y_hats-test_y))

    # Accuracy is 1 minus the error
    accuracy = 1-error

    return accuracy


In [27]:
accuracy = round(test_naive_bayes(x_test, y_test, logprior, loglikelihood),2)
print(f"Naive Bayes accuracy = {accuracy}" )

Naive Bayes accuracy = 0.87
